In [1]:
import string
import pandas as pd
import re
import tensorflow as tf
from pickle import dump
from unicodedata import normalize
import numpy as np
from numpy import array
from numpy.random import rand
from numpy.random import shuffle
import collections
import tensorflow.contrib.legacy_seq2seq as seq2seq
import sys
import babel
import matplotlib.pyplot as plt
%matplotlib inline
import copy

In [2]:
pairs = []
all_lines=[]
source = []
target = []

fptr = open('deu.txt', 'r', encoding='utf-8')
# read all text
lines = fptr.readlines()
#print(lines)
for line in lines :
    #print(line)
    line=line.strip()
    line = re.sub(r"\.", "", line)
    line = re.sub(r"\?", " ?", line)
    line = re.sub(r"\!", "", line)
    line = re.sub(r"\,", " ,", line)
    k = line.split('\t')
    source.append(k[0])
    target.append(k[1])
    all_lines.append(line)
    pairs.append(k)
print(pairs[0:2])
print(all_lines[0:2])
print(source[0:2])
print(target[0:2])

#print(len(source))
source = source[0:5000]
target = target[0:5000]

[['Hi', 'Hallo'], ['Hi', 'Grüß Gott']]
['Hi\tHallo', 'Hi\tGrüß Gott']
['Hi', 'Hi']
['Hallo', 'Grüß Gott']


In [3]:
# Find the length of lines
lengths_source = []
for line in source:
    lengths_source.append(len(line.split()))

# Create a dataframe so that the values can be inspected
lengths_source = pd.DataFrame(lengths_source, columns=['counts'])

In [4]:
lengths_source.describe()

,counts
count,5000.00000
mean,2.77340
std,0.72647
min,1.00000
25%,2.00000
50%,3.00000
75%,3.00000
max,5.00000


In [5]:
print(np.percentile(lengths_source, 95))

4.0


In [6]:
# Limit the input text we will use to the shorter 95%.
max_line_length_eng = 5

text_eng = []
for line in source:
    text_eng.append(line)

In [7]:
# Create a dictionary for the frequency of the vocabulary
vocab_eng = []
v1=[]
for line in text_eng:
    for word in line.split():
        v1.append(word.lower)
        
vocab_eng = set(v1)
#print(vocab_eng)

In [8]:
print("Size of total vocab English:", len(vocab_eng))

Size of total vocab English: 12210


In [9]:
# Find the length of lines
lengths_target = []
for line in target:
    lengths_target.append(len(line.split()))

# Create a dataframe so that the values can be inspected
lengths_target = pd.DataFrame(lengths_target, columns=['counts'])

In [10]:
lengths_target.describe()

,counts
count,5000.000000
mean,3.323800
std,0.931197
min,1.000000
25%,3.000000
50%,3.000000
75%,4.000000
max,8.000000


In [11]:
print(np.percentile(lengths_source, 95))

4.0


In [12]:
# Limit the input text we will use to the shorter 95%.
max_line_length_ger = 9

text_ger = []
for line in target:
    text_ger.append(line)

In [13]:
# Create a dictionary for the frequency of the vocabulary
vocab_ger = []
v2=[]
for line in text_ger:
    for word in line.split():
        v2.append(word.lower)
        
vocab_ger = set(v2)

In [14]:
print("Size of total vocab German:", len(vocab_ger))

Size of total vocab German: 16049


In [15]:
# In case we want to use a different vocabulary sizes for the source and target text, 
# we can set different threshold values.
# Nonetheless, we will create dictionaries to provide a unique integer for each word.

CODES = {'<PAD>': 0, '<EOS>': 1, '<UNK>': 2, '<GO>': 3 }

source_vocab_to_int = {}

source_vocab_to_int = copy.copy(CODES)

for k,v in enumerate(vocab_eng,len(CODES)):
    source_vocab_to_int[k] = v

        
target_vocab_to_int = {}

target_vocab_to_int = copy.copy(CODES)

for k,v in enumerate(vocab_ger,len(CODES)):
    target_vocab_to_int[k] = v
   
        
print(source_vocab_to_int['<UNK>'])

2


In [16]:
# Create dictionaries to map the unique integers to their respective words.
# i.e. an inverse dictionary for vocab_to_int.
source_int_to_vocab = {v_i: v for v, v_i in source_vocab_to_int.items()}
target_int_to_vocab = {v_i: v for v, v_i in target_vocab_to_int.items()}

In [17]:
# Check the length of the dictionaries.
print(len(source_vocab_to_int))
print(len(source_int_to_vocab))
print(len(target_vocab_to_int))
print(len(target_int_to_vocab))

print(source_vocab_to_int)


12214
12214
16053
16053
{'<PAD>': 0, '<EOS>': 1, '<UNK>': 2, '<GO>': 3, 4: <built-in method lower of str object at 0x000000001DC20228>, 5: <built-in method lower of str object at 0x000000001DC297A0>, 6: <built-in method lower of str object at 0x000000001DC29848>, 7: <built-in method lower of str object at 0x000000001DC298F0>, 8: <built-in method lower of str object at 0x000000001DC2E2D0>, 9: <built-in method lower of str object at 0x000000001DC2E378>, 10: <built-in method lower of str object at 0x000000001DC2E420>, 11: <built-in method lower of str object at 0x000000001DC35260>, 12: <built-in method lower of str object at 0x000000001DC35308>, 13: <built-in method lower of str object at 0x000000001DC35688>, 14: <built-in method lower of str object at 0x000000001DC35730>, 15: <built-in method lower of str object at 0x000000001DC4BFB8>, 16: <built-in method lower of str object at 0x000000001DC68848>, 17: <built-in method lower of str object at 0x000000001DC688F0>, 18: <built-in method low

In [18]:
'''print(len(target))
for i in range(len(target)):
    target[i] += ' <EOS>'
print(target[0])'''

"print(len(target))\nfor i in range(len(target)):\n    target[i] += ' <EOS>'\nprint(target[0])"

In [19]:
print(len(source))
print(len(target))

5000
5000


In [20]:
# Convert the text to integers. 
# Replace any words that are not in the respective vocabulary with <UNK> (unknown)
source_int = []
for line in source:
    sentence = []
    for word in line.split():
        if (word != "") :
            sentence.append(source_vocab_to_int[word])
    source_int.append(sentence)
    
target_int = []
for line in target:
    sentence = []
    for word in line.split():
        if (word != "") :
            sentence.append(target_vocab_to_int[word])
    target_int.append(sentence)
    
print(source[0],target[0])    
print(source_int[15]) # source_text_id
print(target_int[15]) # target_text_id

KeyError: 'Hi'

In [ ]:
# Check the lengths
print(len(source_int))
print(len(target_int))

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int) = (source_int,target_int),(source_vocab_to_int, target_vocab_to_int)

In [ ]:
def enc_dec_model_inputs():
    inputs = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets') 
    
    target_sequence_length = tf.placeholder(tf.int32, [None], name='target_sequence_length')
    max_target_len = tf.reduce_max(target_sequence_length)    
    
    return inputs, targets, target_sequence_length, max_target_len

In [ ]:
def hyperparam_inputs():
    lr_rate = tf.placeholder(tf.float32, name='lr_rate')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    return lr_rate, keep_prob

In [ ]:
def process_decoder_input(target_data, target_vocab_to_int, batch_size):
    """
    Preprocess target data for encoding
    :return: Preprocessed target data
    """
    # get '<GO>' id
    go_id = target_vocab_to_int['<GO>']
    
    after_slice = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    after_concat = tf.concat( [tf.fill([batch_size, 1], go_id), after_slice], 1)
    
    return after_concat

In [ ]:
def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, 
                   source_vocab_size, 
                   encoding_embedding_size):
    """
    :return: tuple (RNN output, RNN state)
    """
    embed = tf.contrib.layers.embed_sequence(rnn_inputs, 
                                             vocab_size=source_vocab_size, 
                                             embed_dim=encoding_embedding_size)
    
    stacked_cells = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.DropoutWrapper(tf.contrib.rnn.LSTMCell(rnn_size), keep_prob) for _ in range(num_layers)])
    
    outputs, state = tf.nn.dynamic_rnn(stacked_cells, 
                                       embed, 
                                       dtype=tf.float32)
    return outputs, state

In [ ]:
def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                         target_sequence_length, max_summary_length, 
                         output_layer, keep_prob):
    """
    Create a training process in decoding layer 
    :return: BasicDecoderOutput containing training logits and sample_id
    """
    dec_cell = tf.contrib.rnn.DropoutWrapper(dec_cell, 
                                             output_keep_prob=keep_prob)
    
    # for only input layer
    helper = tf.contrib.seq2seq.TrainingHelper(dec_embed_input, 
                                               target_sequence_length)
    
    decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, 
                                              helper, 
                                              encoder_state, 
                                              output_layer)

    # unrolling the decoder layer
    outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(decoder, 
                                                      impute_finished=True, 
                                                      maximum_iterations=max_summary_length)
    return outputs

In [ ]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                         end_of_sequence_id, max_target_sequence_length,
                         vocab_size, output_layer, batch_size, keep_prob):
    """
    Create a inference process in decoding layer 
    :return: BasicDecoderOutput containing inference logits and sample_id
    """
    dec_cell = tf.contrib.rnn.DropoutWrapper(dec_cell, 
                                             output_keep_prob=keep_prob)
    
    helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(dec_embeddings, 
                                                      tf.fill([batch_size], start_of_sequence_id), 
                                                      end_of_sequence_id)
    
    decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, 
                                              helper, 
                                              encoder_state, 
                                              output_layer)
    
    outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(decoder, 
                                                      impute_finished=True, 
                                                      maximum_iterations=max_target_sequence_length)
    return outputs

In [ ]:
def decoding_layer(dec_input, encoder_state,
                   target_sequence_length, max_target_sequence_length,
                   rnn_size,
                   num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, decoding_embedding_size):
    """
    Create decoding layer
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    target_vocab_size = len(target_vocab_to_int)
    dec_embeddings = tf.Variable(tf.random_uniform([target_vocab_size, decoding_embedding_size]))
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)
    
    cells = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.LSTMCell(rnn_size) for _ in range(num_layers)])
    
    with tf.variable_scope("decode"):
        output_layer = tf.layers.Dense(target_vocab_size)
        train_output = decoding_layer_train(encoder_state, 
                                            cells, 
                                            dec_embed_input, 
                                            target_sequence_length, 
                                            max_target_sequence_length, 
                                            output_layer, 
                                            keep_prob)

    with tf.variable_scope("decode", reuse=True):
        infer_output = decoding_layer_infer(encoder_state, 
                                            cells, 
                                            dec_embeddings, 
                                            target_vocab_to_int['<GO>'], 
                                            target_vocab_to_int['<EOS>'], 
                                            max_target_sequence_length, 
                                            target_vocab_size, 
                                            output_layer,
                                            batch_size,
                                            keep_prob)

    return (train_output, infer_output)

In [ ]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size,
                  target_sequence_length,
                  max_target_sentence_length,
                  source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence model
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    enc_outputs, enc_states = encoding_layer(input_data, 
                                             rnn_size, 
                                             num_layers, 
                                             keep_prob, 
                                             source_vocab_size, 
                                             enc_embedding_size)
    
    dec_input = process_decoder_input(target_data, 
                                      target_vocab_to_int, 
                                      batch_size)
    
    train_output, infer_output = decoding_layer(dec_input,
                                               enc_states, 
                                               target_sequence_length, 
                                               max_target_sentence_length,
                                               rnn_size,
                                              num_layers,
                                              target_vocab_to_int,
                                              target_vocab_size,
                                              batch_size,
                                              keep_prob,
                                              dec_embedding_size)
    
    return train_output, infer_output

In [ ]:
display_step = 300

epochs = 2
batch_size = 128

rnn_size = 128
num_layers = 3

encoding_embedding_size = 200
decoding_embedding_size = 200

learning_rate = 0.001
keep_probability = 0.5

In [ ]:
train_graph = tf.Graph()

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int) = (source_int,target_int),(source_vocab_to_int, target_vocab_to_int)

max_target_sentence_length = max([len(sentence) for sentence in source_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, target_sequence_length, max_target_sequence_length = enc_dec_model_inputs()
    lr, keep_prob = hyperparam_inputs()
    
    train_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                   targets,
                                                   keep_prob,
                                                   batch_size,
                                                   target_sequence_length,
                                                   max_target_sequence_length,
                                                   len(source_vocab_to_int),
                                                   len(target_vocab_to_int),
                                                   encoding_embedding_size,
                                                   decoding_embedding_size,
                                                   rnn_size,
                                                   num_layers,
                                                   target_vocab_to_int)
    
    training_logits = tf.identity(train_logits.rnn_output, name='logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')
    
    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function - weighted softmax cross entropy
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)

In [ ]:
def pad_sentence_batch(sentence_batch, pad_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int] * (max_sentence - len(sentence)) for sentence in sentence_batch]


def get_batches(sources, targets, batch_size, source_pad_int, target_pad_int):
    """Batch targets, sources, and the lengths of their sentences together"""
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size

        # Slice the right amount for the batch
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]

        # Pad
        pad_sources_batch = np.array(pad_sentence_batch(sources_batch, source_pad_int))
        pad_targets_batch = np.array(pad_sentence_batch(targets_batch, target_pad_int))

        # Need the lengths for the _lengths parameters
        pad_targets_lengths = []
        for target in pad_targets_batch:
            pad_targets_lengths.append(len(target))

        pad_source_lengths = []
        for source in pad_sources_batch:
            pad_source_lengths.append(len(source))

        yield pad_sources_batch, pad_targets_batch, pad_source_lengths, pad_targets_lengths
        


In [ ]:
def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target, logits))

In [ ]:
# Split data to training and validation sets
train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]
valid_source = source_int_text[:batch_size]
valid_target = target_int_text[:batch_size]
(valid_sources_batch, valid_targets_batch, valid_sources_lengths, valid_targets_lengths ) = next(get_batches(valid_source,
                                                                                                             valid_target,
                                                                                                             batch_size,
                                                                                                             source_vocab_to_int['<PAD>'],
                                                                                                             target_vocab_to_int['<PAD>']))  
print(len(train_source[0]))
print(len(train_target[0]))

In [ ]:
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch, sources_lengths, targets_lengths) in enumerate(
                get_batches(train_source, train_target, batch_size,
                            source_vocab_to_int['<PAD>'],
                            target_vocab_to_int['<PAD>'])):

            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 keep_prob: keep_probability})


            if batch_i % display_step == 0 and batch_i > 0:
                batch_train_logits = sess.run(
                    inference_logits,
                    {input_data: source_batch,
                     target_sequence_length: targets_lengths,
                     keep_prob: 1.0})

                batch_valid_logits = sess.run(
                    inference_logits,
                    {input_data: valid_sources_batch,
                     target_sequence_length: valid_targets_lengths,
                     keep_prob: 1.0})

                train_acc = get_accuracy(target_batch, batch_train_logits)
                valid_acc = get_accuracy(valid_targets_batch, batch_valid_logits)

                print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.4f}, Validation Accuracy: {:>6.4f}, Loss: {:>6.4f}'
                      .format(epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))
